In [4]:
!unzip '/content/archive (2).zip'

Archive:  /content/archive (2).zip
  inflating: 0.csv                   
  inflating: 1.csv                   
  inflating: 2.csv                   
  inflating: 3.csv                   


In [5]:
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA

In [6]:
#objective: classify gestures by muscle movement
#Dataset: https://www.kaggle.com/kyr7plus/emg-4

allFiles=['/content/0.csv','/content/1.csv','/content/2.csv','/content/3.csv']

list = []
for file in allFiles:
    read = pd.read_csv(file, header = None)
    list.append(read)
df = pd.concat(list)

In [7]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,26.0,4.0,5.0,8.0,-1.0,-13.0,-109.0,-66.0,-9.0,2.0,...,-28.0,61.0,4.0,8.0,5.0,4.0,-7.0,-59.0,16.0,0
1,-47.0,-6.0,-5.0,-7.0,13.0,-1.0,35.0,-10.0,10.0,-4.0,...,-25.0,47.0,6.0,6.0,5.0,13.0,21.0,111.0,15.0,0
2,-19.0,-8.0,-8.0,-8.0,-21.0,-6.0,-79.0,12.0,0.0,5.0,...,-83.0,7.0,7.0,1.0,-8.0,7.0,21.0,114.0,48.0,0
3,2.0,3.0,0.0,2.0,0.0,22.0,106.0,-14.0,-16.0,-2.0,...,-38.0,-11.0,4.0,7.0,11.0,33.0,39.0,119.0,43.0,0
4,6.0,0.0,0.0,-2.0,-14.0,10.0,-51.0,5.0,7.0,0.0,...,38.0,-35.0,-8.0,2.0,6.0,-13.0,-24.0,-112.0,-69.0,0


In [8]:
print('Number of null values in df: ',df.isna().sum().sum())

Number of null values in df:  0


In [9]:
#the column 64 describes the classes of the gestures (from 0 to 3)

x = df.drop([64],axis=1)
y = df[64]

In [10]:
#applying pca to features, to reduce the dimesionality

pca = PCA(n_components=40) #resizing for 40 components
x_pca = pca.fit_transform(x)

In [11]:
#showing the variance of the chosen components
print(f'Total information retained from data (on each Component): \n{pca.explained_variance_ratio_}')
print(f'Percentage of total representation: {sum(pca.explained_variance_ratio_)}')


Total information retained from data (on each Component): 
[0.07220325 0.07008469 0.06072792 0.05484404 0.05300409 0.04713719
 0.04599854 0.04455005 0.03924296 0.03047954 0.0295903  0.02870001
 0.0276893  0.02628102 0.02435048 0.02381102 0.0181809  0.01758357
 0.01697782 0.01515197 0.01468179 0.01386247 0.01362829 0.01285717
 0.01235655 0.01125177 0.01108888 0.01070569 0.01049752 0.01023475
 0.00876117 0.00851693 0.00788317 0.00785001 0.00776865 0.00742114
 0.00690878 0.0066864  0.00658581 0.00620673]
Percentage of total representation: 0.9423423366003134


In [12]:
#validation

skfold = StratifiedKFold(n_splits=5, shuffle=True)

model = GradientBoostingClassifier(n_estimators=250, min_samples_leaf=2, max_depth=5)

result = cross_val_score(model,x_pca,y,cv = skfold)

In [13]:
print(result.mean())

0.8728370757677979
